In [3]:
from langchain_community.document_loaders import Docx2txtLoader, TextLoader, PyPDFLoader
import os
import inspect


print(inspect.getsourcefile(Docx2txtLoader))
print(inspect.getsourcefile(TextLoader))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_community\document_loaders\word_document.py
d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_community\document_loaders\text.py


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print(inspect.getsourcefile(RecursiveCharacterTextSplitter))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_text_splitters\character.py


In [ ]:
from serpapi import GoogleSearch
SERPAPI_KEY = os.environ.get("SERPAPI_KEY")
params = {
  "engine": "google_light",
  "q": "美国总统是谁，介绍一下他",
  "google_domain": "google.com",
  "hl": "zh-cn",
  "gl": "cn",
  "api_key": SERPAPI_KEY,
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]

In [2]:
organic_results

[{'position': 1,
  'title': '美国总统- 维基百科，自由的百科全书',
  'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F',
  'displayed_link': 'zh.wikipedia.org › zh-cn › 美国总统',
  'snippet': '作为当今世界唯一超级大国的国家元首与政府首脑，其号称“自由世界的领导者”及“世界上最有权力的人”。现任（第47任）美国总统是唐纳德·特朗普，于2025年1月20日上任。 美利坚 ...',
  'sitelinks': {'inline': [{'title': '列表',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E5%88%97%E8%A1%A8'},
    {'title': '美国总统选举',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E9%80%89%E4%B8%BE'},
    {'title': '支持率',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E6%94%AF%E6%8C%81%E7%8E%87'},
    {'title': '总统当选人',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E5%BD%93%E9%80%89%E4%BA%BA'}]}},
 {'position': 2,
  'title': '唐纳德·特朗普_百度百科',
  'link': 'https://baike.baidu.com/item/%E5%94%90%E7%BA%B3%E5%BE%B7%C2%B7%E7%89%B9%E6%9C%97%E6%99%AE/9916449',
  'dis

In [ ]:
from typing import Iterable, List, Tuple
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 文本切分
def split_text(documents: Iterable[Document], chunk_size=200, chunk_overlap=30) -> List[Tuple[Document, int, int]]:
    '''
        输入：list[Document]，切分参数
        输出：包含每个分块及其在原始文档中页数和段落位置的列表
    '''
    splliter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    all_chunks = []

    for i, doc in enumerate(documents):
        text = doc.page_content
        source = doc.metadata.get("source", "")
        page_number = doc.metadata.get("page_number", 1)  # 假设文档默认从第1页开始
        chunks = splliter.split_text(text)
        start = 0

        for chunk in chunks:
            # 找到当前分块在原始文档中的结束位置
            end = text.find(chunk, start) + len(chunk)
            # 创建包含分块及其位置信息的元组
            chunk_with_metadata = (Document(page_content=chunk, metadata={
                "source": source,
                "page_number": page_number,
                "chunk_start": start,
                "chunk_end": end,
                "paragraph_number": get_paragraph_number(text, start)
            }), start, end)
            all_chunks.append(chunk_with_metadata)
            # 更新起始位置
            start = end - chunk_overlap

    print(f"分块数：{len(all_chunks)}")
    return all_chunks

def get_paragraph_number(text: str, start: int) -> int:
    '''
        根据起始位置找到分块所在的段落数
    '''
    lines = text.split('\n')
    paragraph_number = 0
    current_position = 0

    for line in lines:
        if line.strip():  # 只计非空行作为段落
            paragraph_number += 1
        if current_position <= start < current_position + len(line):
            return paragraph_number
        current_position += len(line)

    return paragraph_number

# 示例文档
documents = [
    Document(page_content="这是第一页的第一个段落。\n这是第一页的第二个段落。\n这是第一页的第三个段落。", metadata={"source": "doc1.txt", "page_number": 1}),
    Document(page_content="这是第二页的第一个段落。\n这是第二页的第二个段落。\n这是第二页的第三个段落。", metadata={"source": "doc1.txt", "page_number": 2}),
]

# 使用函数
chunks_with_positions = split_text(documents)

# 打印结果
for chunk, start, end in chunks_with_positions:
    print(f"文档源：{chunk.metadata.get('source')}，页数：{chunk.metadata.get('page_number')}，段落：{chunk.metadata.get('paragraph_number')}，起始位置：{start}，结束位置：{end}，分块内容：{chunk.page_content}")
